In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/project1/test.pkl
/kaggle/input/project1/val.pkl
/kaggle/input/project1/train.pkl


In [2]:
import keras
from keras.utils import to_categorical
import numpy as np
import os
import pickle as pkl


train_dict = pkl.load(open("/kaggle/input/project1/train.pkl", "rb"))
val_dict = pkl.load(open("/kaggle/input/project1/val.pkl", "rb"))
test_dict = pkl.load(open("/kaggle/input/project1/test.pkl", "rb"))

In [3]:
from itertools import chain

Prepare data

In [4]:
# prepare word vocab and tag vocab

vocab_dict = {'_unk_': 0, '_w_pad_': 1}

for doc in train_dict['word_seq']:
    for word in doc:
        if(word not in vocab_dict):
            vocab_dict[word] = len(vocab_dict)

tag_dict = {'_t_pad_': 0} # add a padding token

for tag_seq in train_dict['tag_seq']:
    for tag in tag_seq:
        if(tag not in tag_dict):
            tag_dict[tag] = len(tag_dict)
word2idx = vocab_dict
idx2word = {v:k for k,v in word2idx.items()}
tag2idx = tag_dict
idx2tag = {v:k for k,v in tag2idx.items()} 

In [5]:
# The maximum length of a sentence is set to 128
max_sent_length = 128

train_tokens = np.array([[word2idx[w] for w in doc] for doc in train_dict['word_seq']])
val_tokens = np.array([[word2idx.get(w, 0) for w in doc] for doc in val_dict['word_seq']])
test_tokens = np.array([[word2idx.get(w, 0) for w in doc] for doc in test_dict['word_seq']])
# print(train_tokens[0])

train_tags = [[tag2idx[t] for t in t_seq] for t_seq in train_dict['tag_seq']]
# print(train_tags[0])
train_tags = np.array([to_categorical(t_seq, num_classes=len(tag_dict)) for t_seq in train_tags])
# print(train_tags[0])

val_tags = [[tag2idx[t] for t in t_seq] for t_seq in val_dict['tag_seq']]
val_tags = np.array([to_categorical(t_seq, num_classes=len(tag_dict)) for t_seq in val_tags])



In [6]:
def calc_accuracy(preds, tags, padding_id="_t_pad_"):
    """
        Input:
            preds (np.narray): (num_data, length_sentence)
            tags  (np.narray): (num_data, length_sentence)
        Output:
            Proportion of correct prediction. The padding tokens are filtered out.
    """
    preds_flatten = preds.flatten()
    tags_flatten = tags.flatten()
    non_padding_idx = np.where(tags_flatten!=padding_id)[0]
    
    return sum(preds_flatten[non_padding_idx]==tags_flatten[non_padding_idx])/len(non_padding_idx)

In [7]:
# Check accuracy on the training set
train_tags_by_idx = np.argmax(train_tags, axis=2)
train_labels = np.array([[idx2tag[p] for p in preds] for preds in train_tags_by_idx])

val_tags_by_idx = np.argmax(val_tags, axis=2)
val_labels = np.array([[idx2tag[p] for p in preds] for preds in val_tags_by_idx])

In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Embedding, Dropout, BatchNormalization, Input, Add, Concatenate,\
    Bidirectional, SimpleRNN, LSTM, GRU, TimeDistributed
from keras.callbacks import ModelCheckpoint, Callback

In [9]:
class valScore(keras.callbacks.Callback):
    def __init__(self, data):
        self.x = data
    def on_epoch_end(self, epoch, logs={}):
        preds = self.model.predict(self.x, batch_size = 150, verbose = 1)
        preds_tags_by_idx = np.argmax(preds, axis=2)
        preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
        c = calc_accuracy(preds_labels, val_labels)
        print("validation accuracy:", c)

In [10]:
val_score = valScore(val_tokens)

In [11]:
num_training_data = train_tokens.shape[0]
sequence_length = train_tokens.shape[1]
vocabulary_size = len(vocab_dict)
num_tags = train_tags.shape[2]

# Double_lstm
    drop = 0.3 / epochs = 60 / batch_size = 150 / embedding_dim = 60 / hidden_size = 120 / adam lr = 0.001

In [ ]:
# training parameters
drop = 0.3
epochs = 60
batch_size = 150
embedding_dim = 60

hidden_size = 120

In [ ]:
def build_RNN(model_type):
    model = Sequential()
    model.add(Input(shape=(sequence_length,), dtype='int32'))
    model.add(Embedding(input_dim=vocabulary_size, 
                    output_dim=embedding_dim, 
                    input_length=sequence_length))
    model.add(Dropout(drop))
    if model_type == "double_bi_lstm":
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
    if model_type == "triple_bi_lstm":
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
    model.add(TimeDistributed(Dense(units=num_tags, activation='softmax')))
    adam = keras.optimizers.Adam(lr=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['categorical_accuracy'])
    return model

In [ ]:
double_bi_lstm = build_RNN('double_bi_lstm')

In [ ]:
print(double_bi_lstm.summary())

print("Traning Model...")
history = double_bi_lstm.fit(
        train_tokens, 
        train_tags, 
        batch_size=batch_size, 
        epochs=epochs,
        verbose=1,
        validation_data=(val_tokens, val_tags),
        callbacks=[val_score])

In [ ]:
double_bi_lstm.save('/kaggle/working/double_bi_lstm.h5')

In [ ]:
preds =(double_bi_lstm.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('double_bi_lstm', c)

# Double_lstm - lr = 0.005
![](http://)    drop = 0.3 / epochs = 60 / batch_size = 150 / embedding_dim = 60 / hidden_size = 120 / lr = 0.005

In [ ]:
def build_RNN2(model_type):
    model = Sequential()
    model.add(Input(shape=(sequence_length,), dtype='int32'))
    model.add(Embedding(input_dim=vocabulary_size, 
                    output_dim=embedding_dim, 
                    input_length=sequence_length))
    model.add(Dropout(drop))
    if model_type == "double_bi_lstm":
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
    if model_type == "triple_bi_lstm":
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
    model.add(TimeDistributed(Dense(units=num_tags, activation='softmax')))
    adam = keras.optimizers.Adam(lr=0.005)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['categorical_accuracy'])
    return model

In [ ]:
double_bi_lstm2 = build_RNN2('double_bi_lstm')

In [ ]:
print(double_bi_lstm2.summary())

print("Traning Model...")
history = double_bi_lstm2.fit(
        train_tokens, 
        train_tags, 
        batch_size=batch_size, 
        epochs=epochs,
        verbose=1,
        validation_data=(val_tokens, val_tags),
        callbacks=[val_score])

In [ ]:
preds =(double_bi_lstm2.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('double_bi_lstm2', c)

# change embedding size = 50 / lr = 0.001

In [ ]:
# training parameters
drop = 0.3
epochs = 60
batch_size = 150
embedding_dim = 50
hidden_size = 120

In [ ]:
double_bi_lstm3 = build_RNN('double_bi_lstm')

In [ ]:
print(double_bi_lstm3.summary())

print("Traning Model...")
history = double_bi_lstm3.fit(
        train_tokens, 
        train_tags, 
        batch_size=batch_size, 
        epochs=epochs,
        verbose=1,
        validation_data=(val_tokens, val_tags),
        callbacks=[val_score])

In [ ]:
preds =(double_bi_lstm3.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('double_bi_lstm', c)

# NO DROP OUT - embed 50, lr 0.001, but no dropout

In [ ]:
# training parameters
epochs = 60
batch_size = 150
embedding_dim = 50
hidden_size = 120

In [ ]:
def build_RNN3(model_type):
    model = Sequential()
    model.add(Input(shape=(sequence_length,), dtype='int32'))
    model.add(Embedding(input_dim=vocabulary_size, 
                    output_dim=embedding_dim, 
                    input_length=sequence_length))
    if model_type == "double_bi_lstm":
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
    if model_type == "triple_bi_lstm":
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
    model.add(TimeDistributed(Dense(units=num_tags, activation='softmax')))
    adam = keras.optimizers.Adam(lr=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['categorical_accuracy'])
    return model

In [ ]:
double_bi_lstm4 = build_RNN3('double_bi_lstm')
print(double_bi_lstm4.summary())
print("Traning Model...")
history = double_bi_lstm4.fit(
        train_tokens, 
        train_tags, 
        batch_size=batch_size, 
        epochs=epochs,
        verbose=1,
        validation_data=(val_tokens, val_tags),
        callbacks=[val_score])

In [ ]:
preds =(double_bi_lstm4.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('double_bi_lstm', c)

# batch size 200 (embed 50, lr 0.001)

In [ ]:
# training parameters
drop = 0.3
epochs = 60
batch_size = 200
embedding_dim = 50
hidden_size = 120

In [ ]:
double_bi_lstm5 = build_RNN('double_bilstm')
print(double_bi_lstm5.summary())
print("Traning Model...")
history = double_bi_lstm5.fit(
        train_tokens, 
        train_tags, 
        batch_size=batch_size, 
        epochs=epochs,
        verbose=1,
        validation_data=(val_tokens, val_tags),
        callbacks=[val_score])

In [ ]:
preds =(double_bi_lstm5.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('double_bi_lstm', c)

# FINAL MODEL
    DROP 0.3 EPOCHS 60 BATCH_SIZE 150 EMBEDDING_SIZE 50 HIDDEN_SIZE 120 LEARNING_RATE 0.005

In [ ]:
# training parameters
drop = 0.25
epochs = 60
batch_size = 150
embedding_size = 60
hidden_size = 120
learning_rate = 0.005

In [ ]:
def build_RNN(model_type):
    model = Sequential()
    model.add(Input(shape=(sequence_length,), dtype='int32'))
    model.add(Embedding(input_dim=vocabulary_size, 
                    output_dim=embedding_size, 
                    input_length=sequence_length))
    model.add(Dropout(drop))
    if model_type == "double_bilstm":
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
    if model_type == "bilstm_lstm":
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
        model.add(LSTM(units=hidden_size, return_sequences=True))
    if model_type == "triple_bilstm":
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
    if model_type == "double_bigru":
        model.add(Bidirectional(GRU(units=hidden_size, return_sequences=True)))
        model.add(Bidirectional(GRU(units=hidden_size, return_sequences=True))) 
    if model_type == "bigru_gru":
        model.add(Bidirectional(GRU(units=hidden_size, return_sequences=True)))
        model.add(GRU(units=hidden_size, return_sequences=True)) 
    if model_type == "double_birnn":
        model.add(Bidirectional(SimpleRNN(units=hidden_size, return_sequences=True))) 
        model.add(Bidirectional(SimpleRNN(units=hidden_size, return_sequences=True)))
    if model_type == "bilstm_bigru":
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
        model.add(Bidirectional(GRU(units=hidden_size, return_sequences=True)))
    model.add(TimeDistributed(Dense(units=num_tags, activation='softmax')))        
    adam = keras.optimizers.Adam(lr=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['categorical_accuracy'])
    return model

In [ ]:
bigru_gru = build_RNN("bigru_gru")
print(bigru_gru.summary())
print("Traning Model...")
history = bigru_gru.fit(
        train_tokens, 
        train_tags, 
        batch_size=batch_size, 
        epochs=epochs,
        verbose=1,
        validation_data=(val_tokens, val_tags),
        callbacks=[val_score])

In [ ]:
preds =(bigru_gru.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('bigru_gru', c)

In [ ]:
double_bilstm = build_RNN('double_bilstm')
print(double_bilstm.summary())
print("Traning Model...")
history = double_bi_lstm2.fit(
        train_tokens, 
        train_tags, 
        batch_size=batch_size, 
        epochs=epochs,
        verbose=1,
        validation_data=(val_tokens, val_tags),
        callbacks=[val_score])

In [ ]:
preds =(double_bilstm.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('double_bilstm', c)

In [ ]:
double_bilstm.save("/kaggle/working/double_bilstm.h5")

In [ ]:
double_bilstm = keras.models.load_model('/kaggle/working/double_bilstm.h5')
# # example.save("/kaggle/working/double_bigru.h5")
# preds =(example.predict(val_tokens))
# preds_tags_by_idx = np.argmax(preds, axis=2)
# preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
# c = calc_accuracy(preds_labels, val_labels)
# print('double_bilstm', c)

In [ ]:
preds =(double_bi_lstm2.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('double_bi_lstm2', c)

In [ ]:
double_bigru = build_RNN('double_bigru')
print(double_bigru.summary())
print("Traning Model...")
history = double_bigru.fit(
        train_tokens, 
        train_tags, 
        batch_size=batch_size, 
        epochs=epochs,
        verbose=1,
        validation_data=(val_tokens, val_tags),
        callbacks=[val_score])

In [ ]:
preds =(double_bigru.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('double_bigru', c)

In [ ]:
double_bigru.save("double_bigru.h5")

In [ ]:
double_bigru = keras.models.load_model('/kaggle/working/double_bigru.h5')
# example.save("/kaggle/working/double_bigru.h5")
preds =(double_bigru.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('double_bi_gru2', c)

In [ ]:
bilstm_lstm = build_RNN('bilstm_lstm')
print(bilstm_lstm.summary())
print("Traning Model...")
history = bilstm_lstm.fit(
        train_tokens, 
        train_tags, 
        batch_size=batch_size, 
        epochs=epochs,
        verbose=1,
        validation_data=(val_tokens, val_tags),
        callbacks=[val_score])

In [ ]:
preds =(bilstm_lstm.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('bilstm_lstm', c)

In [ ]:
bilstm_lstm.save("bilstm_lstm.h5")

In [ ]:
bilstm_lstm = keras.models.load_model('/kaggle/working/bilstm_lstm.h5')
# example.save("/kaggle/working/double_bigru.h5")
preds =(bilstm_lstm.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('bilstm_lstm', c)

In [ ]:
double_birnn = build_RNN('double_birnn')
print(double_birnn.summary())
print("Traning Model...")
history = double_birnn.fit(
        train_tokens, 
        train_tags, 
        batch_size=batch_size, 
        epochs=epochs,
        verbose=1,
        validation_data=(val_tokens, val_tags),
        callbacks=[val_score])

In [ ]:
preds =(double_birnn.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('double_birnn', c)

In [ ]:
double_birnn.save("double_birnn.h5")

In [ ]:
bilstm_bigru = build_RNN('bilstm_bigru')
print(bilstm_bigru.summary())
print("Traning Model...")
history = bilstm_bigru.fit(
        train_tokens, 
        train_tags, 
        batch_size=batch_size, 
        epochs=epochs,
        verbose=1,
        validation_data=(val_tokens, val_tags),
        callbacks=[val_score])

In [ ]:
preds =(bilstm_bigru.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('bilstm_bigru', c)

In [ ]:
bilstm_bigru.save("bilstm_bigru.h5")

# **TRIPLE**

In [ ]:
triple_bi_lstm = build_RNN('triple_bi_lstm')

In [ ]:
print(triple_bi_lstm.summary())

print("Traning Model...")
history = triple_bi_lstm.fit(
        train_tokens, 
        train_tags, 
        batch_size=batch_size, 
        epochs=epochs,
        verbose=1,
        validation_data=(val_tokens, val_tags),
        callbacks=[val_score])

In [ ]:
triple_bi_lstm.save('/kaggle/working/triple_bi_lstm.h5')

In [ ]:
preds =(triple_bi_lstm.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('triple_bi_lstm', c)

# TRY CRF

In [ ]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

In [ ]:
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras.models import load_model

In [ ]:
def build_RNN_CRF(model_type):
    model = Sequential()
    model.add(Input(shape=(sequence_length,), dtype='int32'))
    model.add(Embedding(input_dim=vocabulary_size, 
                    output_dim=embedding_dim, 
                    input_length=sequence_length))
    model.add(Dropout(drop))
    if model_type == "lstm":
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
    model.add(TimeDistributed(Dense(units=num_tags, activation='softmax')))
    adam = keras.optimizers.Adam(lr=0.005)
    crf = CRF(num_tags)
    model.add(crf)
    model.compile(loss=crf.loss_function, optimizer=adam, metrics=[crf.accuracy])
    return model

In [ ]:
lstm_crf_model = build_RNN_CRF('lstm')
history = double_bi_lstm.fit(
        train_tokens, 
        train_tags, 
        batch_size=batch_size, 
        epochs=epochs,
        verbose=1,
        validation_data=(val_tokens, val_tags),
        callbacks=[val_score])

In [ ]:
print(lstm_crf_model.summary())



In [ ]:
# preds =(lstm_crf_model.predict(val_tokens, verbose=1))
# preds_tags_by_idx = np.argmax(preds, axis=2)
# preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
# c = calc_accuracy(preds_labels, val_labels)
# print(c)

In [ ]:
preds = (lstm_crf_model.predict(val_tokens, verbose=1))
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
pred_labels = pred2label(preds)


In [ ]:
print(pred_labels[0])

In [ ]:
c = calc_accuracy(preds_labels, val_labels)

print(c)


In [ ]:
test_preds_numerical = lstm_crf_model.predict(test_tokens)
test_preds_tags_by_idx = np.argmax(test_preds_numerical, axis=2)
test_preds = np.array([[idx2tag[p] for p in preds] for preds in test_preds_tags_by_idx])
print(test_preds_numerical.shape)
print(test_preds[0])

# ensemble

In [ ]:
import numpy as np
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# models = [double_bi_gru, bi_lstm_lstm]
models_input = Input(shape=(sequence_length,))
models_output = keras.layers.average([double_bilstm(models_input), 
                                      double_bigru(models_input), 
                                      bilstm_lstm(models_input)])
ensemble_model = keras.Model(inputs=models_input,
                             outputs=models_output)

In [ ]:
ensemble_model.summary()

In [ ]:
preds =(ensemble_model.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('ensemble', c)

In [ ]:
test_preds_numerical = ensemble_model.predict(test_tokens)
test_preds_tags_by_idx = np.argmax(test_preds_numerical, axis=2) 
test_preds = np.array([[idx2tag[p] for p in preds] for preds in test_preds_tags_by_idx])
print(test_preds.shape)
print(test_preds[0])

In [ ]:
import json
import pandas as pd

df = pd.DataFrame({'id': test_dict["id"],
                   'labels': [json.dumps(np.array(preds).tolist()) for preds in test_preds]})
df.to_csv('test_preds.csv', index=False)

In [ ]:
pd.read_csv("test_preds.csv")

In [ ]:
val_preds_numerical = ensemble_model.predict(val_tokens)
val_preds_tags_by_idx = np.argmax(val_preds_numerical, axis=2) 
val_preds = np.array([[idx2tag[p] for p in preds] for preds in val_preds_tags_by_idx])

import json
import pandas as pd

df = pd.DataFrame({'id': val_dict["id"],
                   'labels': [json.dumps(np.array(preds).tolist()) for preds in val_preds]})
df.to_csv('val_preds.csv', index=False)

In [ ]:
pd.read_csv("val_preds.csv")

In [ ]:
def evaluate(pred_file, ground_file):
    file_dict = pkl.load(open(ground_file, "rb"))
    file_preds = pd.read_csv(pred_file)
    return calc_accuracy(np.array([json.loads(line) for line in file_preds["labels"]]), 
              np.array(file_dict["tag_seq"]))

print("val accuracy", evaluate('val_preds.csv', "/kaggle/input/project1/val.pkl"))

In [12]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Embedding, Dropout, BatchNormalization, Input, Add, Concatenate,\
    Bidirectional, SimpleRNN, LSTM, GRU, TimeDistributed
from keras.callbacks import ModelCheckpoint, Callback

In [45]:
# training parameters
drop = 0.3
epochs = 60
batch_size = 150
embedding_size = 60
hidden_size = 120
learning_rate = 0.005

In [46]:
double_bilstm_2 = build_RNN("double_bilstm")
print(double_bilstm_2.summary())
print("Traning Model...")
history = double_bilstm_2.fit(
        train_tokens, 
        train_tags, 
        batch_size=batch_size, 
        epochs=epochs,
        verbose=1,
        validation_data=(val_tokens, val_tags),
        callbacks=[val_score])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 128, 60)           4936500   
_________________________________________________________________
dropout_3 (Dropout)          (None, 128, 60)           0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 128, 240)          173760    
_________________________________________________________________
bidirectional_6 (Bidirection (None, 128, 240)          346560    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 128, 65)           15665     
Total params: 5,472,485
Trainable params: 5,472,485
Non-trainable params: 0
_________________________________________________________________
None
Traning Model...
Epoch 1/60
20/20 [==============================] - 0s 22ms/steploss: 1.0316 - categoric

Epoch 26/60
20/20 [==============================] - 0s 24ms/steploss: 0.0991 - categorical_
validation accuracy: 0.9117648760450114
158/158 [==============================] - 20s 129ms/step - loss: 0.0991 - categorical_accuracy: 0.9673 - val_loss: 0.3776 - val_categorical_accuracy: 0.9192
Epoch 27/60
20/20 [==============================] - 0s 23ms/steploss: 0.0967 - categorical_ac
validation accuracy: 0.9118748011223929
158/158 [==============================] - 20s 125ms/step - loss: 0.0967 - categorical_accuracy: 0.9680 - val_loss: 0.3823 - val_categorical_accuracy: 0.9193
Epoch 28/60
20/20 [==============================] - 0s 23ms/steploss: 0.0940 - categorical_
validation accuracy: 0.9146923544215916
158/158 [==============================] - 19s 122ms/step - loss: 0.0940 - categorical_accuracy: 0.9689 - val_loss: 0.3777 - val_categorical_accuracy: 0.9219
Epoch 29/60
20/20 [==============================] - 0s 23ms/steploss: 0.0915 - categorical_ac
validation accuracy: 0.9115913

20/20 [==============================] - 0s 24ms/steploss: 0.0606 - categorical_
validation accuracy: 0.9129017327663513
158/158 [==============================] - 20s 126ms/step - loss: 0.0606 - categorical_accuracy: 0.9791 - val_loss: 0.4443 - val_categorical_accuracy: 0.9203
Epoch 55/60
20/20 [==============================] - 0s 23ms/steploss: 0.0604 - categorical_ac
validation accuracy: 0.9119529057826377
158/158 [==============================] - 19s 118ms/step - loss: 0.0604 - categorical_accuracy: 0.9792 - val_loss: 0.4324 - val_categorical_accuracy: 0.9194
Epoch 56/60
20/20 [==============================] - 0s 22ms/steploss: 0.0600 - categorical_ac
validation accuracy: 0.913419537736122
158/158 [==============================] - 19s 123ms/step - loss: 0.0600 - categorical_accuracy: 0.9794 - val_loss: 0.4428 - val_categorical_accuracy: 0.9207
Epoch 57/60
20/20 [==============================] - 0s 22ms/steploss: 0.0593 - categorical_ac
validation accuracy: 0.911284677022766
15

In [47]:
preds =(double_bilstm_2.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('double_bilstm_2', c)

double_bilstm_2 0.9159131013335647


In [48]:
double_bilstm_2.save("/kaggle/working/double_bilstm_2.h5")

In [ ]:
# training parameters
drop = 0.25
epochs = 60
batch_size = 150
embedding_size = 60
hidden_size = 120
learning_rate = 0.005

In [14]:
def build_RNN(model_type):
    model = Sequential()
    model.add(Input(shape=(sequence_length,), dtype='int32'))
    model.add(Embedding(input_dim=vocabulary_size, 
                    output_dim=embedding_size, 
                    input_length=sequence_length))
    model.add(Dropout(drop))
    if model_type == "double_bilstm":
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
    if model_type == "bilstm_lstm":
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
        model.add(LSTM(units=hidden_size, return_sequences=True))
    if model_type == "triple_bilstm":
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
    if model_type == "double_bigru":
        model.add(Bidirectional(GRU(units=hidden_size, return_sequences=True)))
        model.add(Bidirectional(GRU(units=hidden_size, return_sequences=True))) 
    if model_type == "bigru_gru":
        model.add(Bidirectional(GRU(units=hidden_size, return_sequences=True)))
        model.add(GRU(units=hidden_size, return_sequences=True)) 
    if model_type == "double_birnn":
        model.add(Bidirectional(SimpleRNN(units=hidden_size, return_sequences=True))) 
        model.add(Bidirectional(SimpleRNN(units=hidden_size, return_sequences=True)))
    if model_type == "bilstm_bigru":
        model.add(Bidirectional(LSTM(units=hidden_size, return_sequences=True)))
        model.add(Bidirectional(GRU(units=hidden_size, return_sequences=True)))
    model.add(TimeDistributed(Dense(units=num_tags, activation='softmax')))        
    adam = keras.optimizers.Adam(lr=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['categorical_accuracy'])
    return model

In [15]:
double_bilstm = build_RNN("double_bilstm")
print(double_bilstm.summary())
print("Traning Model...")
history = double_bilstm.fit(
        train_tokens, 
        train_tags, 
        batch_size=batch_size, 
        epochs=epochs,
        verbose=1,
        validation_data=(val_tokens, val_tags),
        callbacks=[val_score])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 128, 60)           4936500   
_________________________________________________________________
dropout (Dropout)            (None, 128, 60)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 128, 240)          173760    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128, 240)          346560    
_________________________________________________________________
time_distributed (TimeDistri (None, 128, 65)           15665     
Total params: 5,472,485
Trainable params: 5,472,485
Non-trainable params: 0
_________________________________________________________________
None
Traning Model...
Epoch 1/60
20/20 [==============================] - 0s 23ms/steploss: 1.0603 - categorical

Epoch 26/60
20/20 [==============================] - 0s 23ms/steploss: 0.0907 - categorical_ac
validation accuracy: 0.9133182909543233
158/158 [==============================] - 19s 121ms/step - loss: 0.0907 - categorical_accuracy: 0.9701 - val_loss: 0.3834 - val_categorical_accuracy: 0.9206
Epoch 27/60
20/20 [==============================] - 0s 23ms/steploss: 0.0873 - categorical_ac
validation accuracy: 0.9145940004049872
158/158 [==============================] - 19s 123ms/step - loss: 0.0873 - categorical_accuracy: 0.9713 - val_loss: 0.3858 - val_categorical_accuracy: 0.9218
Epoch 28/60
20/20 [==============================] - 0s 23ms/steploss: 0.0845 - categorical_ac
validation accuracy: 0.9133443258410715
158/158 [==============================] - 19s 121ms/step - loss: 0.0845 - categorical_accuracy: 0.9720 - val_loss: 0.3928 - val_categorical_accuracy: 0.9207
Epoch 29/60
20/20 [==============================] - 0s 23ms/steploss: 0.0812 - categorical_ac
validation accuracy: 0.914

20/20 [==============================] - 0s 23ms/steploss: 0.0470 - categorical_ac
validation accuracy: 0.912467817987214
158/158 [==============================] - 19s 118ms/step - loss: 0.0470 - categorical_accuracy: 0.9839 - val_loss: 0.4763 - val_categorical_accuracy: 0.9199
Epoch 55/60
20/20 [==============================] - 0s 22ms/steploss: 0.0464 - categorical_ac
validation accuracy: 0.9124822818131852
158/158 [==============================] - 20s 124ms/step - loss: 0.0464 - categorical_accuracy: 0.9840 - val_loss: 0.4683 - val_categorical_accuracy: 0.9199
Epoch 56/60
20/20 [==============================] - 0s 22ms/steploss: 0.0461 - categorical_ac
validation accuracy: 0.9133530041366542
158/158 [==============================] - 18s 115ms/step - loss: 0.0461 - categorical_accuracy: 0.9841 - val_loss: 0.4682 - val_categorical_accuracy: 0.9207
Epoch 57/60
20/20 [==============================] - 0s 23ms/steploss: 0.0452 - categorical_ac
validation accuracy: 0.9138939512279788

In [20]:
preds =(double_bilstm.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('double_bilstm', c)

double_bilstm 0.9129393387138766


In [58]:
double_bigru2 = build_RNN('double_bigru')
print(double_bigru2.summary())
print("Traning Model...")
history = double_bigru2.fit(
        train_tokens, 
        train_tags, 
        batch_size=batch_size, 
        epochs=epochs,
        verbose=1,
        validation_data=(val_tokens, val_tags),
        callbacks=[val_score])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 128, 60)           4936500   
_________________________________________________________________
dropout_4 (Dropout)          (None, 128, 60)           0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 128, 240)          131040    
_________________________________________________________________
bidirectional_8 (Bidirection (None, 128, 240)          260640    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 128, 65)           15665     
Total params: 5,343,845
Trainable params: 5,343,845
Non-trainable params: 0
_________________________________________________________________
None
Traning Model...
Epoch 1/60
20/20 [==============================] - 0s 21ms/steploss: 0.7902 - categoric

Epoch 26/60
20/20 [==============================] - 0s 21ms/steploss: 0.0922 - categorical_ac
validation accuracy: 0.9165524024414938
158/158 [==============================] - 18s 114ms/step - loss: 0.0922 - categorical_accuracy: 0.9688 - val_loss: 0.3799 - val_categorical_accuracy: 0.9236
Epoch 27/60
20/20 [==============================] - 0s 22ms/steploss: 0.0904 - categorical_ac
validation accuracy: 0.9156932511788018
158/158 [==============================] - 19s 123ms/step - loss: 0.0904 - categorical_accuracy: 0.9694 - val_loss: 0.3818 - val_categorical_accuracy: 0.9228
Epoch 28/60
20/20 [==============================] - 0s 21ms/steploss: 0.0896 - categorical_ac
validation accuracy: 0.9139807341838063
158/158 [==============================] - 20s 125ms/step - loss: 0.0896 - categorical_accuracy: 0.9696 - val_loss: 0.3842 - val_categorical_accuracy: 0.9212
Epoch 29/60
20/20 [==============================] - 0s 21ms/steploss: 0.0882 - categorical_ac
validation accuracy: 0.916

Epoch 54/60
20/20 [==============================] - 0s 21ms/steploss: 0.0684 - categorical_ac
validation accuracy: 0.9170731001764587
158/158 [==============================] - 19s 122ms/step - loss: 0.0684 - categorical_accuracy: 0.9765 - val_loss: 0.4263 - val_categorical_accuracy: 0.9241
Epoch 55/60
20/20 [==============================] - 0s 22ms/steploss: 0.0676 - categorical_ac
validation accuracy: 0.916158986375076
158/158 [==============================] - 19s 121ms/step - loss: 0.0676 - categorical_accuracy: 0.9768 - val_loss: 0.4258 - val_categorical_accuracy: 0.9232
Epoch 56/60
20/20 [==============================] - 0s 21ms/steploss: 0.0669 - categorical_ac
validation accuracy: 0.9165292603199399
158/158 [==============================] - 19s 121ms/step - loss: 0.0669 - categorical_accuracy: 0.9770 - val_loss: 0.4290 - val_categorical_accuracy: 0.9236
Epoch 57/60
20/20 [==============================] - 0s 21ms/steploss: 0.0659 - categorical_ac
validation accuracy: 0.9171

In [59]:
preds =(double_bigru2.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('double_bigru2', c)

double_bigru2 0.9163007318695942


In [21]:
double_bigru = build_RNN('double_bigru')
print(double_bigru.summary())
print("Traning Model...")
history = double_bigru.fit(
        train_tokens, 
        train_tags, 
        batch_size=batch_size, 
        epochs=epochs,
        verbose=1,
        validation_data=(val_tokens, val_tags),
        callbacks=[val_score])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 128, 60)           4936500   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 60)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128, 240)          131040    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128, 240)          260640    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 128, 65)           15665     
Total params: 5,343,845
Trainable params: 5,343,845
Non-trainable params: 0
_________________________________________________________________
None
Traning Model...
Epoch 1/60
20/20 [==============================] - 0s 21ms/steploss: 0.8309 - categoric

Epoch 26/60
20/20 [==============================] - 0s 22ms/steploss: 0.0755 - categorical_ac
validation accuracy: 0.9173710549914663
158/158 [==============================] - 18s 112ms/step - loss: 0.0755 - categorical_accuracy: 0.9743 - val_loss: 0.4108 - val_categorical_accuracy: 0.9244
Epoch 27/60
20/20 [==============================] - 0s 22ms/steploss: 0.0739 - categorical_ac
validation accuracy: 0.9163267667563424
158/158 [==============================] - 17s 109ms/step - loss: 0.0739 - categorical_accuracy: 0.9748 - val_loss: 0.4116 - val_categorical_accuracy: 0.9234
Epoch 28/60
20/20 [==============================] - 0s 22ms/steploss: 0.0723 - categorical_ac
validation accuracy: 0.9175619774942868
158/158 [==============================] - 18s 115ms/step - loss: 0.0723 - categorical_accuracy: 0.9754 - val_loss: 0.4123 - val_categorical_accuracy: 0.9245
Epoch 29/60
20/20 [==============================] - 0s 21ms/steploss: 0.0708 - categorical_ac
validation accuracy: 0.918

Epoch 54/60
20/20 [==============================] - 0s 23ms/steploss: 0.0524 - categorical_ac
validation accuracy: 0.9167780381266453
158/158 [==============================] - 18s 113ms/step - loss: 0.0524 - categorical_accuracy: 0.9819 - val_loss: 0.4669 - val_categorical_accuracy: 0.9238
Epoch 55/60
20/20 [==============================] - 0s 22ms/steploss: 0.0525 - categorical_ac
validation accuracy: 0.9167086117619833
158/158 [==============================] - 18s 117ms/step - loss: 0.0525 - categorical_accuracy: 0.9819 - val_loss: 0.4684 - val_categorical_accuracy: 0.9237
Epoch 56/60
20/20 [==============================] - 0s 22ms/steploss: 0.0520 - categorical_ac
validation accuracy: 0.9183227747403743
158/158 [==============================] - 18s 116ms/step - loss: 0.0520 - categorical_accuracy: 0.9821 - val_loss: 0.4726 - val_categorical_accuracy: 0.9252
Epoch 57/60
20/20 [==============================] - 0s 22ms/steploss: 0.0519 - categorical_ac
validation accuracy: 0.915

In [22]:
preds =(double_bigru.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('double_bigru', c)

double_bigru 0.9175648702594811


In [23]:
double_bigru.save("/kaggle/working/double_bigru.h5")

In [24]:
bilstm_lstm = build_RNN('bilstm_lstm')
print(bilstm_lstm.summary())
print("Traning Model...")
history = bilstm_lstm.fit(
        train_tokens, 
        train_tags, 
        batch_size=batch_size, 
        epochs=epochs,
        verbose=1,
        validation_data=(val_tokens, val_tags),
        callbacks=[val_score])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 128, 60)           4936500   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128, 60)           0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 128, 240)          173760    
_________________________________________________________________
lstm_3 (LSTM)                (None, 128, 120)          173280    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 128, 65)           7865      
Total params: 5,291,405
Trainable params: 5,291,405
Non-trainable params: 0
_________________________________________________________________
None
Traning Model...
Epoch 1/60
20/20 [==============================] - 0s 17ms/steploss: 1.0228 - categoric

Epoch 26/60
20/20 [==============================] - 0s 19ms/steploss: 0.0981 - categorical_ac
validation accuracy: 0.9111458242934422
158/158 [==============================] - 16s 102ms/step - loss: 0.0981 - categorical_accuracy: 0.9679 - val_loss: 0.3899 - val_categorical_accuracy: 0.9187
Epoch 27/60
20/20 [==============================] - 0s 17ms/steploss: 0.0950 - categorical_ac
validation accuracy: 0.9078741068587463
158/158 [==============================] - 16s 101ms/step - loss: 0.0950 - categorical_accuracy: 0.9690 - val_loss: 0.3984 - val_categorical_accuracy: 0.9157
Epoch 28/60
20/20 [==============================] - 0s 17ms/steploss: 0.0927 - categorical_ac
validation accuracy: 0.9116057739593277
158/158 [==============================] - 17s 108ms/step - loss: 0.0927 - categorical_accuracy: 0.9695 - val_loss: 0.4010 - val_categorical_accuracy: 0.9191
Epoch 29/60
20/20 [==============================] - 0s 17ms/steploss: 0.0900 - categorical_ac
validation accuracy: 0.909

Epoch 54/60
20/20 [==============================] - 0s 18ms/steploss: 0.0582 - categorical_ac
validation accuracy: 0.9106974456883334
158/158 [==============================] - 17s 108ms/step - loss: 0.0582 - categorical_accuracy: 0.9803 - val_loss: 0.4671 - val_categorical_accuracy: 0.9182
Epoch 55/60
20/20 [==============================] - 0s 17ms/steploss: 0.0578 - categorical_ac
validation accuracy: 0.910795799704938
158/158 [==============================] - 15s 98ms/step - loss: 0.0578 - categorical_accuracy: 0.9803 - val_loss: 0.4730 - val_categorical_accuracy: 0.9183
Epoch 56/60
20/20 [==============================] - 0s 17ms/steploss: 0.0567 - categorical_ac
validation accuracy: 0.911215250658104
158/158 [==============================] - 16s 100ms/step - loss: 0.0567 - categorical_accuracy: 0.9807 - val_loss: 0.4669 - val_categorical_accuracy: 0.9187
Epoch 57/60
20/20 [==============================] - 0s 17ms/steploss: 0.0568 - categorical_ac
validation accuracy: 0.913630

In [25]:
preds =(bilstm_lstm.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('bilstm_lstm', c)

bilstm_lstm 0.9122768954843935


In [26]:
bilstm_lstm.save("bilstm_lstm.h5")

In [49]:
models_input = Input(shape=(sequence_length,))
models_output = keras.layers.average([double_bilstm_2(models_input), 
                                      double_bigru(models_input), 
                                      bilstm_lstm(models_input)])
ensemble_model = keras.Model(inputs=models_input,
                             outputs=models_output)

In [28]:
ensemble_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 128, 65)      5472485     input_4[0][0]                    
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 128, 65)      5343845     input_4[0][0]                    
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 128, 65)      5291405     input_4[0][0]                    
_______________________________________________________________________________________

In [50]:
preds =(ensemble_model.predict(val_tokens))
preds_tags_by_idx = np.argmax(preds, axis=2)
preds_labels = np.array([[idx2tag[p] for p in pred] for pred in preds_tags_by_idx])
c = calc_accuracy(preds_labels, val_labels)
print('ensemble_model', c)

ensemble_model 0.9242471578581967


In [51]:
test_preds_numerical = ensemble_model.predict(test_tokens)
test_preds_tags_by_idx = np.argmax(test_preds_numerical, axis=2) 
test_preds = np.array([[idx2tag[p] for p in preds] for preds in test_preds_tags_by_idx])
print(test_preds.shape)
print(test_preds[0])

(2950, 128)
['O' 'O' 'IMMUNE_RESPONSE' 'IMMUNE_RESPONSE' 'O' 'ORGANISM' 'O' 'O' 'O'
 'DISEASE_OR_SYNDROME' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'CHEMICAL'
 'CHEMICAL' 'O' 'LIVESTOCK' 'O' 'CELL' 'O' 'O' 'O' 'O' 'O' 'O' 'O'
 'ORGANISM' 'O' 'ORGANISM' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O'
 'CHEMICAL' 'O' 'DISEASE_OR_SYNDROME' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O'
 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O'
 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'IMMUNE_RESPONSE'
 'IMMUNE_RESPONSE' 'O' 'ORG' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O'
 'GENE_OR_GENOME' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O'
 'CHEMICAL' 'O' 'O' 'O' 'O' 'CELL_FUNCTION' 'O' 'O' 'O' 'O' 'O'
 'CELL_COMPONENT' 'O' 'O']


In [52]:
# Let's take the baseline 1 as an example, where we predict all labels as 1.

import json
import pandas as pd

df = pd.DataFrame({'id': test_dict["id"],
                   'labels': [json.dumps(np.array(preds).tolist()) for preds in test_preds]})
df.to_csv('test_preds.csv', index=False)

In [53]:
pd.read_csv("test_preds.csv")

,id,labels
0,0,"[""O"", ""O"", ""IMMUNE_RESPONSE"", ""IMMUNE_RESPONSE..."
1,1,"[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""_t_pad_"", ""_t_..."
2,2,"[""ORG"", ""ORG"", ""O"", ""RESEARCH_ACTIVITY"", ""RESE..."
3,3,"[""O"", ""O"", ""O"", ""CHEMICAL"", ""CHEMICAL"", ""O"", ""..."
4,4,"[""O"", ""CHEMICAL"", ""CHEMICAL"", ""CHEMICAL"", ""O"",..."
...,...,...
2945,2945,"[""DATE"", ""O"", ""CORONAVIRUS"", ""O"", ""O"", ""O"", ""O..."
2946,2946,"[""VIRUS"", ""CHEMICAL"", ""CHEMICAL"", ""CHEMICAL"", ..."
2947,2947,"[""O"", ""O"", ""O"", ""CHEMICAL"", ""O"", ""O"", ""DISEASE..."
2948,2948,"[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""GENE_OR_G..."


In [54]:
val_preds_numerical = ensemble_model.predict(val_tokens)
val_preds_tags_by_idx = np.argmax(val_preds_numerical, axis=2) 
val_preds = np.array([[idx2tag[p] for p in preds] for preds in val_preds_tags_by_idx])

import json
import pandas as pd

df = pd.DataFrame({'id': val_dict["id"],
                   'labels': [json.dumps(np.array(preds).tolist()) for preds in val_preds]})
df.to_csv('val_preds.csv', index=False)

In [55]:
pd.read_csv("val_preds.csv")

,id,labels
0,0,"[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""GENE_OR_GENOME..."
1,1,"[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ..."
2,2,"[""GPE"", ""O"", ""O"", ""O"", ""DISEASE_OR_SYNDROME"", ..."
3,3,"[""GENE_OR_GENOME"", ""GENE_OR_GENOME"", ""O"", ""VIR..."
4,4,"[""EUKARYOTE"", ""DISEASE_OR_SYNDROME"", ""O"", ""O"",..."
...,...,...
2945,2945,"[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""GENE..."
2946,2946,"[""O"", ""O"", ""O"", ""IMMUNE_RESPONSE"", ""IMMUNE_RES..."
2947,2947,"[""O"", ""DATE"", ""CORONAVIRUS"", ""O"", ""O"", ""O"", ""O..."
2948,2948,"[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ..."


In [56]:
def evaluate(pred_file, ground_file):
    file_dict = pkl.load(open(ground_file, "rb"))
    file_preds = pd.read_csv(pred_file)
    return calc_accuracy(np.array([json.loads(line) for line in file_preds["labels"]]),
                  np.array(file_dict["tag_seq"]))

In [60]:
print("val accuracy", evaluate('val_preds.csv', "/kaggle/input/project1/val.pkl"))

val accuracy 0.9242471578581967


In [18]:
double_bilstm.save("/kaggle/working/double_bilstm.h5")